In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import gensim
nltk.download('punkt')
nltk.download('stopwords')
import numpy as np
#nltk.download()


C:\Users\Manohar Battula\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package punkt to C:\Users\Manohar
[nltk_data]     Battula\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Manohar
[nltk_data]     Battula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data_train = pd.read_csv("information_train.csv", sep = '\t')
train = pd.read_csv("train.csv")
data_test = pd.read_csv("information_test.csv", sep = '\t')
test = pd.read_csv("test.csv")
sample_sub = pd.read_csv("sample_submission.csv")

In [3]:
data_test = data_test.rename(columns= {'set': 'article_set'})

In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
abstract         3522 non-null object
article_title    3522 non-null object
author_str       3089 non-null object
pmid             3522 non-null int64
pub_date         3522 non-null object
set              3522 non-null int64
full_Text        141 non-null object
dtypes: int64(2), object(5)
memory usage: 192.7+ KB


In [5]:
data_test.head()

,abstract,article_title,author_str,pmid,pub_date,article_set,full_Text
0,Cell lines selected in multiple steps for incr...,"The gene for a novel protein, a member of the ...","M M Chaudhuri, P N Tonin, W H Lewis, P R Srini...",1311171,1992-02-01,17,NaN
1,Prolyl 4-hydroxylase (EC 1.14.11.2) is an esse...,Inhibition of prolyl 4-hydroxylase by hydroxya...,"C J Cunliffe, T J Franklin",3028370,1986-10-15,17,NaN
2,From the structure-activity relationships of k...,Time-dependent inactivation of chick-embryo pr...,"V Gunzler, H M Hanauske-Abel, R Myllyla, J Moh...",3036081,1987-02-15,17,NaN
3,The anthracyclines doxorubicin and daunorubici...,Syncatalytic inactivation of prolyl 4-hydroxyl...,"V Gunzler, H M Hanauske-Abel, R Myllyla, D D K...",2840891,1988-04-15,17,NaN
4,The levels of lysine hydroxylase protein and t...,Minoxidil specifically decreases the expressio...,"T Hautala, J Heikkinen, K I Kivirikko, R Myllyla",1314568,1992-04-01,17,NaN


In [6]:
import datetime
data_test['pub_date'] = pd.to_datetime(data_test['pub_date'])
data_test.head()

,abstract,article_title,author_str,pmid,pub_date,article_set,full_Text
0,Cell lines selected in multiple steps for incr...,"The gene for a novel protein, a member of the ...","M M Chaudhuri, P N Tonin, W H Lewis, P R Srini...",1311171,1992-02-01,17,NaN
1,Prolyl 4-hydroxylase (EC 1.14.11.2) is an esse...,Inhibition of prolyl 4-hydroxylase by hydroxya...,"C J Cunliffe, T J Franklin",3028370,1986-10-15,17,NaN
2,From the structure-activity relationships of k...,Time-dependent inactivation of chick-embryo pr...,"V Gunzler, H M Hanauske-Abel, R Myllyla, J Moh...",3036081,1987-02-15,17,NaN
3,The anthracyclines doxorubicin and daunorubici...,Syncatalytic inactivation of prolyl 4-hydroxyl...,"V Gunzler, H M Hanauske-Abel, R Myllyla, D D K...",2840891,1988-04-15,17,NaN
4,The levels of lysine hydroxylase protein and t...,Minoxidil specifically decreases the expressio...,"T Hautala, J Heikkinen, K I Kivirikko, R Myllyla",1314568,1992-04-01,17,NaN


In [7]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2034 entries, 0 to 2033
Data columns (total 7 columns):
abstract         2034 non-null object
article_title    2034 non-null object
author_str       1881 non-null object
pmid             2034 non-null int64
pub_date         2034 non-null datetime64[ns]
article_set      2034 non-null int64
full_Text        26 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 111.3+ KB


In [8]:
data_test.article_set.value_counts()

9     358
4     336
10    324
17    266
15    239
19    209
1     168
11     71
7      63
Name: article_set, dtype: int64

In [9]:
data_test_copy = data_test.copy()

In [10]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df



In [11]:
standardize_text(data_test_copy, "article_title")
standardize_text(data_test_copy, "abstract")

,abstract,article_title,author_str,pmid,pub_date,article_set,full_Text
0,cell lines selected in multiple steps for incr...,"the gene for a novel protein, a member of the ...","M M Chaudhuri, P N Tonin, W H Lewis, P R Srini...",1311171,1992-02-01,17,NaN
1,prolyl 4 hydroxylase (ec 1 14 11 2) is an esse...,inhibition of prolyl 4 hydroxylase by hydroxya...,"C J Cunliffe, T J Franklin",3028370,1986-10-15,17,NaN
2,from the structure activity relationships of k...,time dependent inactivation of chick embryo pr...,"V Gunzler, H M Hanauske-Abel, R Myllyla, J Moh...",3036081,1987-02-15,17,NaN
3,the anthracyclines doxorubicin and daunorubici...,syncatalytic inactivation of prolyl 4 hydroxyl...,"V Gunzler, H M Hanauske-Abel, R Myllyla, D D K...",2840891,1988-04-15,17,NaN
4,the levels of lysine hydroxylase protein and t...,minoxidil specifically decreases the expressio...,"T Hautala, J Heikkinen, K I Kivirikko, R Myllyla",1314568,1992-04-01,17,NaN
5,1 the number of reactive thiol groups in mamm...,the reactivities and ionization properties of ...,"H C Hawkins, R B Freedman",2025220,1991-04-15,17,NaN
6,1 the activities of protein disulphide isomer...,comparison of the activities of protein disulp...,"H C Hawkins, E C Blackburn, R B Freedman",2025222,1991-04-15,17,NaN
7,prolyl 4 hydroxylase was partially purified an...,characterization of a low relative molecular m...,"D D Kaska, V Gunzler, K I Kivirikko, R Myllyla",3036064,1987-01-15,17,NaN
8,the deacetylcephalosporin c synthetase (hydrox...,cloning and sequencing of the beta lactam hydr...,"S Kovacevic, J R Miller",1987130,1991-01-01,17,NaN
9,the microsomal triacylglycerol transfer protei...,the role of protein disulphide isomerase in th...,"A Lamberg, M Jauhiainen, J Metso, C Ehnholm, C...",8615825,1996-04-15,17,NaN


In [12]:
def preprocessing(df,text_field):
    df['tokens'] = df[text_field]
    df['tokens'] = df.tokens.str.split()
    stop = stopwords.words('english')
    df['tokens'] = df['tokens'].apply(lambda x: [item for item in x if item not in stop])
    df['tokens'] = df['tokens'].apply(lambda x: [porter.stem(y) for y in x])
    return df


In [13]:
porter = nltk.PorterStemmer()
def get_similarities(df):  # pass text_field_byset
    dic = {}
    dic_tokens = {}
    for x in df.article_set.unique():
        final_tokens = [token for token in df.loc[df.article_set == x].tokens] 
        dictionary = gensim.corpora.Dictionary(final_tokens)
        corpus = [dictionary.doc2bow(final_tokens) for final_tokens in final_tokens]
        dic_tokens[x] = final_tokens
    
        tf_idf = gensim.models.TfidfModel(corpus)
        sims = gensim.similarities.Similarity('C:/Users/Manohar Battula/Desktop/Innoplexus Hack',tf_idf[corpus],
                                      num_features=len(dictionary))
        similarities = sims[tf_idf[corpus]]
        dic[x]= similarities
    return dic


In [14]:
def top5(sim_dic,art_set):
    top_docs = sim_dic[art_set][1].argsort()[::-1][:5]
    top_docs_with_score = [(index, round(sim_dic[art_set][1][index], 3))
                                for index in top_docs]
    print(top_docs_with_score)

In [15]:
## Absrtact Model
standardize_text(data_test_copy, 'abstract')
abstract_byset = data_test_copy[['abstract','article_set']].copy()
preprocessing(abstract_byset,'abstract')
abstract_dic= get_similarities(abstract_byset)

In [16]:
## Title Model 
standardize_text(data_test_copy, 'article_title')
title_byset = data_test_copy[['article_title','article_set']].copy()
preprocessing(title_byset,'article_title')
title_dic= get_similarities(title_byset)

In [17]:
def get_pred_dfs(df,sim_dic1, sim_dic2): 
    
    ### df --> data_test                                                                                 
    #pred =[]
    
    pmid_top = pd.DataFrame(np.zeros((0,10)), \
                            columns=['top1','top2','top3','top4','top5','top6','top7','top8','top9','top10'])
    date_top = pd.DataFrame(np.zeros((0,10)),\
                            columns=['dtop1','dtop2','dtop3','dtop4','dtop5','dtop6','dtop7','dtop8','dtop9','dtop10'])
    
    for i in df.article_set.unique():
        pmid_list_sets = df.loc[df.article_set == i].pmid.tolist()
        date_list_sets = df.loc[df.article_set == i].pub_date.tolist()
        
        combined_sim_list = (np.array(sim_dic1[i]) + np.array(sim_dic2[i])).tolist()
        df_pmid = pd.DataFrame(combined_sim_list, columns = pmid_list_sets)
        df_date = pd.DataFrame(combined_sim_list, columns = date_list_sets)
        x = df_pmid.T
        y = df_date.T
        
        for j in x.columns:
            df1row = pd.DataFrame(x.nlargest(10, j).index.tolist(), \
                                  index=['top1','top2','top3','top4','top5','top6','top7','top8','top9','top10']).T
            pmid_top = pd.concat([pmid_top.astype(int), df1row], axis=0)
            #pmid_top= pmid_top.drop(['top1'], axis=1)
            #restolist = pmid_top.values.tolist()
        
        for j in y.columns:
            df1row = pd.DataFrame(y.nlargest(10, j).index.tolist(),\
                                  index=['dtop1','dtop2','dtop3','dtop4','dtop5','dtop6','dtop7','dtop8','dtop9','dtop10']).T
            date_top = pd.concat([date_top, df1row], axis=0)
            #date_top= date_top.drop(['top1'], axis=1)
            #restolist = date_top.values.tolist()
        #for k in range(0,len(restolist)):
            #pred.append(restolist[k])
    return pmid_top,date_top


In [18]:
## combined model date 
pmid_top , date_top = get_pred_dfs(data_test, abstract_dic, title_dic)

In [19]:
def get_prediction(pmid_top,date_top):
    
    pred =[]
    for i in range(0,len(pmid_top)):
        x=[]
        for j in range(0,date_top.shape[1]):
            if ((date_top.iloc[i][0] > date_top.iloc[i][j]) and len(x)<3):
                y = pmid_top.iloc[i][j] 
                x.append(y) 
        pred.append(x)
    preddf = pd.DataFrame()
    preddf['pmid'] = data_test.pmid
    preddf['ref_list'] = pred
    prediction = pd.merge(test,preddf, on='pmid', how='left')
    return prediction

In [20]:
solution = get_prediction(pmid_top,date_top)

In [21]:
'''
from sklearn.feature_extraction.text import TfidfVectorizer

def get_similarities_tfidf(df,text_field):     # pass data_test
    dic = {}
    #dic_tokens = {}
    for x in df.article_set.unique():
        vectorizer = TfidfVectorizer(max_df=0.3, lowercase = True, ngram_range= (1,3),
                             min_df=1, stop_words='english')
                             
        
        corpus = [y for y in df.loc[df.article_set == x][text_field]]
        dtm = vectorizer.fit_transform(corpus)
        tfidf_sims = gensim.similarities.Similarity('C:/Users/Manohar Battula/Desktop/Innoplexus Hack',
                                                    dtm,num_features=dtm.shape[1])
        similarities = tfidf_sims[dtm]
        dic[x]= similarities
    return dic

title_dic_tfidf = get_similarities_tfidf(data_test,'article_title')
abstract_dic_tfidf = get_similarities_tfidf(data_test,'abstract')
'''

"\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\ndef get_similarities_tfidf(df,text_field):     # pass data_test\n    dic = {}\n    #dic_tokens = {}\n    for x in df.article_set.unique():\n        vectorizer = TfidfVectorizer(max_df=0.3, lowercase = True, ngram_range= (1,3),\n                             min_df=1, stop_words='english')\n                             \n        \n        corpus = [y for y in df.loc[df.article_set == x][text_field]]\n        dtm = vectorizer.fit_transform(corpus)\n        tfidf_sims = gensim.similarities.Similarity('C:/Users/Manohar Battula/Desktop/Innoplexus Hack',\n                                                    dtm,num_features=dtm.shape[1])\n        similarities = tfidf_sims[dtm]\n        dic[x]= similarities\n    return dic\n\ntitle_dic_tfidf = get_similarities_tfidf(data_test,'article_title')\nabstract_dic_tfidf = get_similarities_tfidf(data_test,'abstract')\n"

In [22]:
'''
### Lsi Model

from gensim import corpora, models, similarities
def get_similarities_lsi(df):  # pass article_title_byset
    dic = {}
    dic_tokens = {}
    for x in df.article_set.unique():
        final_tokens = [token for token in df.loc[df.article_set == x].tokens] 
        dictionary = gensim.corpora.Dictionary(final_tokens)
        corpus = [dictionary.doc2bow(final_tokens) for final_tokens in final_tokens]
        dic_tokens[x] = final_tokens
    
        tf_idf = gensim.models.TfidfModel(corpus)
        #corpus_tfidf = tf_idf[corpus]
        lsi = models.LsiModel(corpus)
        sims = gensim.similarities.MatrixSimilarity(lsi[corpus])
                                                   
                                      
        similarities = sims[lsi[corpus]]
        dic[x]= similarities
    return dic
    
title_dic_lsi = get_similarities_lsi(article_title_byset)
abstract_dic_lsi = get_similarities_lsi(abstract_byset)
'''

'\n### Lsi Model\n\nfrom gensim import corpora, models, similarities\ndef get_similarities_lsi(df):  # pass article_title_byset\n    dic = {}\n    dic_tokens = {}\n    for x in df.article_set.unique():\n        final_tokens = [token for token in df.loc[df.article_set == x].tokens] \n        dictionary = gensim.corpora.Dictionary(final_tokens)\n        corpus = [dictionary.doc2bow(final_tokens) for final_tokens in final_tokens]\n        dic_tokens[x] = final_tokens\n    \n        tf_idf = gensim.models.TfidfModel(corpus)\n        #corpus_tfidf = tf_idf[corpus]\n        lsi = models.LsiModel(corpus)\n        sims = gensim.similarities.MatrixSimilarity(lsi[corpus])\n                                                   \n                                      \n        similarities = sims[lsi[corpus]]\n        dic[x]= similarities\n    return dic\n    \ntitle_dic_lsi = get_similarities_lsi(article_title_byset)\nabstract_dic_lsi = get_similarities_lsi(abstract_byset)\n'

In [23]:
'''

### word2vec

from gensim.models import Word2Vec
preprocessing(article_title_byset,'article_title')
sentences = [token for token in title_byset.tokens]

word2vec_model = Word2Vec(sentences, min_count = 1)

def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.vocab]
    return np.mean(word2vec_model[doc], axis=0)

from sklearn.metrics.pairwise import cosine_similarity
sims = cosine_similarity(np.array([document_vector(word2vec_model, doc)
                                                       for doc in sentences]))

def get_similarities_word2vec(df):  # EX. pass title_byset, abstract_byset
    dic = {}
    dic_tokens = {}
    sentences = [token for token in df.tokens]
    for x in df.article_set.unique():
        corpus = [token for token in df.loc[df.article_set == x].tokens] 
        word2vec_model = Word2Vec(sentences, min_count = 1)
        sims = cosine_similarity(np.array([document_vector(word2vec_model, doc)
                                                       for doc in corpus]))
        dic[x]= sims
    return dic

title_dic_w2v = get_similarities_word2vec(title_byset)
abstract_dic_w2v = get_similarities_word2vec(abstract_byset)
'''

"\n\n### word2vec\n\nfrom gensim.models import Word2Vec\npreprocessing(article_title_byset,'article_title')\nsentences = [token for token in title_byset.tokens]\n\nword2vec_model = Word2Vec(sentences, min_count = 1)\n\ndef document_vector(word2vec_model, doc):\n    # remove out-of-vocabulary words\n    doc = [word for word in doc if word in word2vec_model.wv.vocab]\n    return np.mean(word2vec_model[doc], axis=0)\n\nfrom sklearn.metrics.pairwise import cosine_similarity\nsims = cosine_similarity(np.array([document_vector(word2vec_model, doc)\n                                                       for doc in sentences]))\n\ndef get_similarities_word2vec(df):  # EX. pass title_byset, abstract_byset\n    dic = {}\n    dic_tokens = {}\n    sentences = [token for token in df.tokens]\n    for x in df.article_set.unique():\n        corpus = [token for token in df.loc[df.article_set == x].tokens] \n        word2vec_model = Word2Vec(sentences, min_count = 1)\n        sims = cosine_similarity(np